In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-p6/train_data.csv
/kaggle/input/2021-ml-p6/test_data.csv
/kaggle/input/2021-ml-p6/sample_submit.csv


### data load

In [32]:
# data load
xy_data = pd.read_csv('/kaggle/input/2021-ml-p6/train_data.csv')
print(xy_data.head())
test_data = pd.read_csv('/kaggle/input/2021-ml-p6/test_data.csv')
print(test_data.head())
submit = pd.read_csv('/kaggle/input/2021-ml-p6/sample_submit.csv')
print(submit.head())

   ID  company      model  year  transmission  mileage  fueltype  tax   mpg  \
0   0        4      Auris  2015             1    47541         4  145  46.3   
1   1        0         X2  2018             3     5000         0  145  50.4   
2   2        2      Focus  2017             3    47018         4  145  51.4   
3   3        2      Focus  2020             1     1550         4  145  49.6   
4   4        0   3 Series  2017             3    23505         0  145  64.2   

   engineSize  price  
0         1.6   8095  
1         2.0  21726  
2         1.0  10490  
3         1.0  17490  
4         2.0  18995  
   ID  company      model  year  transmission  mileage  fueltype  tax   mpg  \
0   0        2        Ka+  2017             1    10150         4  145  57.7   
1   1        2       Kuga  2013             1    50216         0  145  53.3   
2   2        3         Q5  2016             1    34189         0  200  47.9   
3   3        1        I10  2020             1     1900         4  145  

In [33]:
# x_train / test data
x_data = xy_data.drop(['ID', 'price'], axis=1) # ID label은 필요없는 label이고, price label은 y값이므로 drop!
print(x_data.head())
test_data = test_data.drop('ID', axis=1) # ID label은 필요없는 label이므로 drop!
print(test_data.head())

   company      model  year  transmission  mileage  fueltype  tax   mpg  \
0        4      Auris  2015             1    47541         4  145  46.3   
1        0         X2  2018             3     5000         0  145  50.4   
2        2      Focus  2017             3    47018         4  145  51.4   
3        2      Focus  2020             1     1550         4  145  49.6   
4        0   3 Series  2017             3    23505         0  145  64.2   

   engineSize  
0         1.6  
1         2.0  
2         1.0  
3         1.0  
4         2.0  
   company      model  year  transmission  mileage  fueltype  tax   mpg  \
0        2        Ka+  2017             1    10150         4  145  57.7   
1        2       Kuga  2013             1    50216         0  145  53.3   
2        3         Q5  2016             1    34189         0  200  47.9   
3        1        I10  2020             1     1900         4  145  56.5   
4        0   3 Series  2014             0    79809         0   20  68.9   

  

In [34]:
# y_train
y_data = xy_data['price']
print(y_data.head())

0     8095
1    21726
2    10490
3    17490
4    18995
Name: price, dtype: int64


### data label

In [35]:
# train data의 model이 실수가 아니므로 label encoder
from sklearn.preprocessing import LabelEncoder

classle = LabelEncoder()
classle.fit(x_data['model'].values) # 하나의 변환기를 통해 변환해주기 위해 train data에서 fit!

train_le = classle.transform(x_data['model'].values) # 위에 fit한 것으로 똑같이 변환(transform)
test_le = classle.transform(test_data['model'].values) # 위에 fit한 것으로 똑같이 변환(transform) (중요!! fit_transform 아님!!)

x_data['model'] = train_le
test_data['model'] = test_le

print(x_data.head())
print(test_data.head())

   company  model  year  transmission  mileage  fueltype  tax   mpg  \
0        4     15  2015             1    47541         4  145  46.3   
1        0     87  2018             3     5000         0  145  50.4   
2        2     26  2017             3    47018         4  145  51.4   
3        2     26  2020             1     1550         4  145  49.6   
4        0      2  2017             3    23505         0  145  64.2   

   engineSize  
0         1.6  
1         2.0  
2         1.0  
3         1.0  
4         2.0  
   company  model  year  transmission  mileage  fueltype  tax   mpg  \
0        2     44  2017             1    10150         4  145  57.7   
1        2     46  2013             1    50216         0  145  53.3   
2        3     60  2016             1    34189         0  200  47.9   
3        1     34  2020             1     1900         4  145  56.5   
4        0      2  2014             0    79809         0   20  68.9   

   engineSize  
0         1.2  
1         2.0  
2 

### 학습 : KNN regression

In [25]:
from sklearn.neighbors import KNeighborsRegressor

regressor = KNeighborsRegressor(n_neighbors = 3, weights = 'distance')

regressor.fit(x_data, y_data) # 학습

test_pred = regressor.predict(test_data)

### 학습 : KNN regression - standard 사용

In [28]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(x_data) # train data로 먼저 fit(표준화) 해주고
X_data_std = sc.transform(x_data) # 위에 fit한 것으로 transform 해주기!!
X_test_std = sc.transform(test_data) # 이것 또한 위에 fit한 것으로 transform 해줘야 함!! (주의!! train data에 표준화 했으면 test data에도 해주기!!)

print(x_data.head())
X_data_std[1:5,]

   company  model  year  transmission  mileage  fueltype  tax   mpg  \
0        4     15  2015             1    47541         4  145  46.3   
1        0     87  2018             3     5000         0  145  50.4   
2        2     26  2017             3    47018         4  145  51.4   
3        2     26  2020             1     1550         4  145  49.6   
4        0      2  2017             3    23505         0  145  64.2   

   engineSize  
0         1.6  
1         2.0  
2         1.0  
3         1.0  
4         2.0  


array([[-1.51026792,  1.96991151,  0.47662902,  1.83851219, -0.87994672,
        -1.18757679,  0.40725992, -0.32237328,  0.54882437],
       [ 0.03447068, -0.26065734,  0.01252285,  1.83851219,  1.0740513 ,
         0.89248979,  0.40725992, -0.26845529, -1.12125567],
       [ 0.03447068, -0.26065734,  1.40484136, -0.18376245, -1.04038494,
         0.89248979,  0.40725992, -0.36550768, -1.12125567],
       [-1.51026792, -1.1382582 ,  0.01252285,  1.83851219, -0.0193933 ,
        -1.18757679,  0.40725992,  0.42169506,  0.54882437]])

In [29]:
from sklearn.neighbors import KNeighborsRegressor

regressor = KNeighborsRegressor(n_neighbors = 3, weights = 'distance')

regressor.fit(X_data_std, y_data) # 학습 - 위에 표준화 해준 것으로 학습하기!!

test_pred = regressor.predict(X_test_std) # 예측 - 위에 표준화 해준 것으로 예측!!

### 학습 : KNN regression - normalization 사용

In [36]:
from sklearn.preprocessing import Normalizer

nm = Normalizer()
nm.fit(x_data) # train data로 먼저 fit(정규화) 해주고
X_data_nm = sc.transform(x_data) # 위에 fit한 것으로 transform 해주기!!
X_test_nm = sc.transform(test_data) # 이것 또한 위에 fit한 것으로 transform 해줘야 함!! (주의!! train data에 정규화 했으면 test data에도 해주기!!)

print(x_data.head())
X_data_nm[1:5,]

   company  model  year  transmission  mileage  fueltype  tax   mpg  \
0        4     15  2015             1    47541         4  145  46.3   
1        0     87  2018             3     5000         0  145  50.4   
2        2     26  2017             3    47018         4  145  51.4   
3        2     26  2020             1     1550         4  145  49.6   
4        0      2  2017             3    23505         0  145  64.2   

   engineSize  
0         1.6  
1         2.0  
2         1.0  
3         1.0  
4         2.0  


array([[-1.51026792,  1.96991151,  0.47662902,  1.83851219, -0.87994672,
        -1.18757679,  0.40725992, -0.32237328,  0.54882437],
       [ 0.03447068, -0.26065734,  0.01252285,  1.83851219,  1.0740513 ,
         0.89248979,  0.40725992, -0.26845529, -1.12125567],
       [ 0.03447068, -0.26065734,  1.40484136, -0.18376245, -1.04038494,
         0.89248979,  0.40725992, -0.36550768, -1.12125567],
       [-1.51026792, -1.1382582 ,  0.01252285,  1.83851219, -0.0193933 ,
        -1.18757679,  0.40725992,  0.42169506,  0.54882437]])

In [37]:
from sklearn.neighbors import KNeighborsRegressor

regressor = KNeighborsRegressor(n_neighbors = 3, weights = 'distance')

regressor.fit(X_data_nm, y_data) # 학습 - 위에 정규화 해준 것으로 학습하기!!

test_pred = regressor.predict(X_test_nm) # 예측 - 위에 정규화 해준 것으로 예측!!

### submit에 넣기

In [38]:
submit['price'] = test_pred
submit = submit.astype(np.int64)
submit.to_csv('submit.csv', mode='w', header=True, index=False)